In [1]:
import pandas as pd

# Load the downloaded taxi trip data
print("Loading Parquet file...")
df_trips = pd.read_parquet('yellow_tripdata_2024-01.parquet')
print("✅ File loaded successfully!")

# Display the first 5 rows to see the columns
print("\nFirst 5 rows of data:")
display(df_trips.head())

# Display a summary of the data (columns, memory usage, etc.)
print("\nData summary:")
df_trips.info()

Loading Parquet file...
✅ File loaded successfully!

First 5 rows of data:


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0



Data summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2964624 entries, 0 to 2964623
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport

In [2]:
# Load the taxi zone lookup data
df_zones = pd.read_csv('taxi+_zone_lookup.csv')

# Display the first 5 rows to see what's inside
print("Taxi Zone Lookup Data (first 5 rows):")
display(df_zones.head())

Taxi Zone Lookup Data (first 5 rows):


,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [3]:
# --- Clean up and merge the data ---

# We only need a few columns for our logistics problem. Let's select them.
df_trips_cleaned = df_trips[['tpep_pickup_datetime', 'PULocationID']].copy()

# Rename columns for clarity
df_trips_cleaned.rename(columns={
    'tpep_pickup_datetime': 'Timestamp',
    'PULocationID': 'LocationID'
}, inplace=True)

# --- Merge the trip data with the zone lookup data ---
print("Merging trip data with zone information...")

# This command joins the two tables based on the 'LocationID'
df_merged = pd.merge(
    left=df_trips_cleaned,
    right=df_zones,
    how='inner',
    on='LocationID'
)

print("✅ Merge successful!")

# Display the first 5 rows of the newly combined data
print("\nCombined Data (first 5 rows):")
display(df_merged.head())

# See how many orders we have in each borough
print("\nOrder count by Borough:")
print(df_merged['Borough'].value_counts())

Merging trip data with zone information...
✅ Merge successful!

Combined Data (first 5 rows):


,Timestamp,LocationID,Borough,Zone,service_zone
0,2024-01-01 00:57:55,186,Manhattan,Penn Station/Madison Sq West,Yellow Zone
1,2024-01-01 00:03:00,140,Manhattan,Lenox Hill East,Yellow Zone
2,2024-01-01 00:17:06,236,Manhattan,Upper East Side North,Yellow Zone
3,2024-01-01 00:36:38,79,Manhattan,East Village,Yellow Zone
4,2024-01-01 00:46:51,211,Manhattan,SoHo,Yellow Zone



Order count by Borough:
Borough
Manhattan        2646948
Queens            273128
Brooklyn           25258
Unknown            10360
Bronx               6905
EWR                  295
Staten Island         72
Name: count, dtype: int64


In [4]:
import pandas as pd
import io

# --- Raw CSV data for NYC Taxi Zone coordinates ---
# This data is now embedded in our code to avoid any download errors.
csv_data = """LocationID,latitude,longitude
1,40.644990,-74.174230
2,40.675690,-73.800100
3,40.864470,-73.847420
4,40.723750,-73.981420
5,40.562860,-74.184470
6,40.596820,-74.126290
7,40.744110,-73.992010
8,40.627760,-73.951230
9,40.835430,-73.864330
10,40.652070,-73.945830
11,40.628880,-73.994870
12,40.588520,-73.963160
13,40.708870,-74.009380
14,40.668240,-73.980390
15,40.837330,-73.902260
16,40.764170,-73.961540
17,40.692690,-73.951560
18,40.695160,-73.922440
19,40.679610,-73.904830
20,40.648370,-74.017550
21,40.627270,-74.077330
22,40.671160,-73.943110
23,40.785340,-73.942450
24,40.791730,-73.944220
25,40.718530,-73.953560
26,40.641320,-73.955930
27,40.671190,-73.868720
28,40.743140,-73.974440
29,40.706850,-74.015240
30,40.545620,-74.166820
31,40.864820,-73.916100
32,40.852860,-73.896790
33,40.671810,-73.969180
34,40.686560,-73.981880
35,40.694860,-73.974280
36,40.711280,-73.994130
37,40.725830,-73.999060
38,40.887320,-73.910320
39,40.803770,-73.872720
40,40.820980,-73.889320
41,40.575910,-74.139090
42,40.612030,-74.025340
43,40.752690,-73.984280
44,40.714090,-73.942120
45,40.718450,-73.986870
46,40.738180,-74.002590
47,40.709320,-73.996120
48,40.768070,-73.982740
49,40.760160,-73.991250
50,40.757070,-73.990130
51,40.618030,-74.004230
52,40.740620,-73.913230
53,40.741710,-73.866330
54,40.796590,-73.839690
55,40.807750,-73.856980
56,40.689360,-73.899320
57,40.693150,-73.882420
58,40.840240,-73.856090
59,40.871140,-73.834140
60,40.896310,-73.854830
61,40.705290,-73.938390
62,40.748360,-73.953680
63,40.709350,-73.961290
64,40.761890,-73.826130
65,40.732890,-74.006930
66,40.707740,-73.951540
67,40.700010,-73.924230
68,40.761760,-73.958500
69,40.785070,-73.815560
70,40.814520,-73.910580
71,40.729990,-73.988020
72,40.696140,-73.994780
73,40.793830,-73.961560
74,40.799010,-73.968130
75,40.811740,-73.952230
76,40.702730,-74.014280
77,40.685370,-74.001220
78,40.658810,-73.974950
79,40.726480,-73.980590
80,40.695990,-73.969980
81,40.669560,-73.891890
82,40.679120,-73.961220
83,40.686030,-73.961910
84,40.635840,-73.884580
85,40.672580,-73.914270
86,40.687530,-73.906730
87,40.748640,-73.987730
88,40.759290,-73.985160
89,40.583610,-74.093950
90,40.763190,-73.991800
91,40.603330,-73.943990
92,40.598070,-74.076590
93,40.631890,-73.965560
94,40.599010,-74.062820
95,40.778840,-73.953530
96,40.784420,-73.949740
97,40.730470,-73.958810
98,40.608030,-74.088010
99,40.592750,-74.198390
100,40.751140,-73.992240
101,40.644120,-74.005180
102,40.638570,-73.994270
103,40.771980,-73.874430
104,40.775830,-73.869970
105,40.781060,-73.850540
106,40.728830,-73.978830
107,40.733610,-73.994230
108,40.751740,-73.800050
109,40.755450,-73.820520
110,40.735870,-73.791530
111,40.605280,-74.065870
112,40.612490,-73.978180
113,40.759160,-73.969240
114,40.764520,-73.978010
115,40.852170,-73.844910
116,40.743130,-73.992840
117,40.687050,-73.873230
118,40.835560,-73.850680
119,40.790070,-73.976050
120,40.738870,-73.976310
121,40.810330,-73.829040
122,40.825630,-73.811550
123,40.796390,-73.814230
124,40.790930,-73.853240
125,40.748970,-73.998930
126,40.767570,-73.805360
127,40.742970,-74.006590
128,40.735390,-73.993140
129,40.787620,-73.842040
130,40.759590,-73.844450
131,40.753340,-73.872970
132,40.774100,-73.980600
133,40.804890,-73.938830
134,40.755790,-73.883490
135,40.760030,-73.921350
136,40.766330,-73.858790
137,40.771230,-73.987540
138,40.718030,-73.843450
139,40.701150,-73.828550
140,40.771120,-73.957580
141,40.776930,-73.962240
142,40.767660,-73.971840
143,40.769930,-73.958560
144,40.743320,-73.982290
145,40.825130,-73.940340
146,40.827760,-73.931220
147,40.865730,-73.830490
148,40.755490,-73.976410
149,40.798650,-73.931180
150,40.815910,-73.940730
151,40.742300,-74.001600
152,40.745670,-73.993810
153,40.739480,-74.002450
154,40.746060,-73.811020
155,40.791550,-73.932720
156,40.873240,-73.865970
157,40.810560,-73.889370
158,40.749940,-73.987100
159,40.857640,-73.918970
160,40.868090,-73.899630
161,40.757040,-73.978270
162,40.763320,-73.973950
163,40.765970,-73.973210
164,40.761020,-73.968320
165,40.729090,-73.805560
166,40.739780,-73.990420
167,40.596040,-74.156640
168,40.795490,-73.940980
169,40.817360,-73.956960
170,40.756280,-73.962630
171,40.692370,-73.833330
172,40.584340,-74.106880
173,40.706110,-73.911380
174,40.692880,-73.867950
175,40.659910,-73.820980
176,40.601450,-74.098710
177,40.690850,-73.998010
178,40.680430,-73.993420
179,40.645480,-73.959380
180,40.652110,-73.964240
181,40.737110,-73.986300
182,40.844320,-73.880120
183,40.866160,-73.882700
184,40.826720,-73.859660
185,40.810000,-73.896780
186,40.752010,-73.993210
187,40.638760,-74.084260
188,40.669810,-73.929840
189,40.678460,-73.931750
190,40.757870,-73.969890
191,40.595930,-74.088670
192,40.583560,-74.072710
193,40.767120,-73.881240
194,40.697420,-73.808020
195,40.635870,-73.812970
196,40.709110,-73.867090
197,40.710730,-73.880480
198,40.724730,-73.880890
199,40.873530,-73.844160
200,40.659340,-73.796330
201,40.602730,-74.113330
202,40.715340,-74.004310
203,40.640690,-73.944230
204,40.551720,-74.204360
205,40.678520,-73.842780
206,40.642510,-73.848690
207,40.627040,-74.156090
208,40.771030,-73.905910
209,40.739330,-73.974460
210,40.730720,-73.991240
211,40.721930,-74.000160
212,40.686530,-73.834010
213,40.672040,-73.791520
214,40.584740,-74.057390
215,40.617540,-74.043810
216,40.617430,-74.092170
217,40.626880,-74.114780
218,40.573220,-74.098370
219,40.596660,-74.045790
220,40.577280,-74.067820
221,40.607480,-74.153920
222,40.655270,-73.991910
223,40.688140,-73.910300
224,40.749170,-74.001200
225,40.686250,-73.939160
226,40.710490,-73.983940
227,40.697620,-73.910790
228,40.758410,-73.919250
229,40.782340,-73.953790
230,40.760010,-73.988020
231,40.792290,-73.959930
232,40.797230,-73.951520
233,40.733560,-73.985930
234,40.749760,-73.991380
235,40.853040,-73.828230
236,40.775980,-73.953680
237,40.771820,-73.966230
238,40.754230,-73.973490
239,40.757900,-73.978250
240,40.840740,-73.939230
241,40.864750,-73.897440
242,40.860010,-73.871330
243,40.803520,-73.949600
244,40.807870,-73.945410
245,40.893120,-73.882340
246,40.740290,-73.994010
247,40.676380,-73.975390
248,40.669020,-73.961430
249,40.747480,-73.986220
250,40.728950,-73.992080
251,40.570000,-74.113060
252,40.851970,-73.928170
253,40.858700,-73.933330
254,40.876540,-73.909940
255,40.743990,-73.945930
256,40.734410,-73.954490
257,40.748350,-73.944370
258,40.718320,-73.908510
259,40.670350,-73.834920
260,40.702780,-73.883640
261,40.733310,-73.959830
262,40.753330,-73.982270
263,40.748360,-73.988180
264,0.000000,0.000000
265,0.000000,0.000000
"""

# --- Load the coordinates data from the embedded string ---
print("Loading coordinates data from embedded string...")
# Use io.StringIO to treat the string as a file
df_coords = pd.read_csv(io.StringIO(csv_data))
print("✅ Coordinates loaded successfully!")

# --- Perform the final merge to add coordinates ---
print("\nMerging data with coordinates...")

# Merge our main data with the coordinates based on the 'LocationID'
df_final = pd.merge(
    left=df_merged,
    right=df_coords[['LocationID', 'latitude', 'longitude']], # Select only the columns we need
    how='inner',
    on='LocationID'
)

# --- Clean up invalid locations ---
# Remove any orders where the location is (0,0) or unknown
df_final = df_final[df_final['latitude'] != 0].copy()
df_final = df_final[df_final['Borough'] != 'Unknown'].copy()


print("✅ Final merge successful!")
print("\nOur data now has latitude and longitude for every order.")

# Display the first 5 rows of the final dataset
display(df_final.head())

# Display the final shape of our dataset
print(f"\nShape of the final dataset: {df_final.shape}")

Loading coordinates data from embedded string...
✅ Coordinates loaded successfully!

Merging data with coordinates...
✅ Final merge successful!

Our data now has latitude and longitude for every order.


,Timestamp,LocationID,Borough,Zone,service_zone,latitude,longitude
0,2024-01-01 00:57:55,186,Manhattan,Penn Station/Madison Sq West,Yellow Zone,40.75201,-73.99321
1,2024-01-01 00:03:00,140,Manhattan,Lenox Hill East,Yellow Zone,40.77112,-73.95758
2,2024-01-01 00:17:06,236,Manhattan,Upper East Side North,Yellow Zone,40.77598,-73.95368
3,2024-01-01 00:36:38,79,Manhattan,East Village,Yellow Zone,40.72648,-73.98059
4,2024-01-01 00:46:51,211,Manhattan,SoHo,Yellow Zone,40.72193,-74.00016



Shape of the final dataset: (2952606, 7)


In [5]:
import pandas as pd
import numpy as np
import plotly.express as px

# --- 1. Recreate Warehouses as Medical Depots ---
# We'll create this DataFrame directly in the code for simplicity
warehouse_data = {
    'Warehouse_ID': ['DEPOT_MANHATTAN', 'DEPOT_QUEENS', 'DEPOT_BROOKLYN', 'DEPOT_BRONX', 'DEPOT_STATEN_ISLAND'],
    'Latitude': [40.7831, 40.7282, 40.6782, 40.8448, 40.5795],
    'Longitude': [-73.9712, -73.7949, -73.9442, -73.8648, -74.1502],
    'Sterile_Kits_Stock': np.random.randint(500, 2000, size=5),
    'Vaccine_Packs_Stock': np.random.randint(500, 2000, size=5)
}
warehouses_df = pd.DataFrame(warehouse_data)


# --- 2. Prepare a Sample of Clinic/Pharmacy Orders ---
print("Taking a random sample of 200 clinic/pharmacy orders...")
orders_df = df_final.sample(n=200, random_state=42).copy()

# Add simulated medical supply demand
orders_df['Sterile_Kits_Demand'] = np.random.randint(10, 50, size=len(orders_df))
orders_df['Vaccine_Packs_Demand'] = np.random.randint(10, 50, size=len(orders_df))

# Reset the index to get a clean Order_ID
orders_df.reset_index(drop=True, inplace=True)
orders_df['Order_ID'] = orders_df.index
print("✅ Sampled orders are ready.")


# --- 3. Create the Updated Interactive Map ---
print("\nGenerating interactive map of medical distribution network...")

fig = px.scatter_mapbox(
    orders_df,
    lat="latitude",
    lon="longitude",
    hover_name="Zone",
    hover_data=["Order_ID", "Sterile_Kits_Demand", "Vaccine_Packs_Demand"],
    color_discrete_sequence=["blue"],
    zoom=9,
    height=600,
    title="Medical Supply Network: Depots and Clinic/Pharmacy Orders"
)

# Add a second layer for our medical depots
fig.add_trace(
    px.scatter_mapbox(
        warehouses_df,
        lat="Latitude",
        lon="Longitude",
        hover_name="Warehouse_ID",
        hover_data=["Sterile_Kits_Stock", "Vaccine_Packs_Stock"],
        color_discrete_sequence=["red"],
        size_max=20,
    ).data[0]
)

fig.update_layout(
    mapbox_style="open-street-map",
    margin={"r":0,"t":40,"l":0,"b":0}
)

fig.show()

Taking a random sample of 200 clinic/pharmacy orders...
✅ Sampled orders are ready.

Generating interactive map of medical distribution network...


C:\Users\abravikumar\AppData\Local\Temp\ipykernel_15072\2328559095.py:34: DeprecationWarning: *scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/
  fig = px.scatter_mapbox(
C:\Users\abravikumar\AppData\Local\Temp\ipykernel_15072\2328559095.py:48: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [6]:
import numpy as np

# --- Combine depots and orders into a single list of locations ---
# The first 5 locations will be our depots, the rest are orders
all_locations_df = pd.concat([
    warehouses_df.rename(columns={'Latitude': 'lat', 'Longitude': 'lon'}),
    orders_df.rename(columns={'latitude': 'lat', 'longitude': 'lon'})
], ignore_index=True)

# --- Function to calculate distance (a simple approximation) ---
# This uses the Pythagorean theorem on a scaled version of lat/lon
def calculate_distance(lat1, lon1, lat2, lon2):
    # Scaling factor for NYC area (approximate)
    # This is a simplification; for real-world use, you'd use a road network API
    lat_scale = 69 
    lon_scale = 55
    x1, y1 = lat1 * lat_scale, lon1 * lon_scale
    x2, y2 = lat2 * lat_scale, lon2 * lon_scale
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# --- Create the distance matrix ---
print("Creating the distance matrix...")
num_locations = len(all_locations_df)
distance_matrix = np.zeros((num_locations, num_locations))

for i in range(num_locations):
    for j in range(num_locations):
        if i == j:
            continue # Distance from a point to itself is 0
        loc_i = all_locations_df.iloc[i]
        loc_j = all_locations_df.iloc[j]
        distance_matrix[i, j] = calculate_distance(loc_i['lat'], loc_i['lon'], loc_j['lat'], loc_j['lon'])

print("✅ Distance matrix created successfully!")
print(f"Shape of the matrix: {distance_matrix.shape}")
print("\nFirst 5x5 part of the matrix (distances between depots):")
print(distance_matrix[:5, :5].round(2))

Creating the distance matrix...
✅ Distance matrix created successfully!
Shape of the matrix: (205, 205)

First 5x5 part of the matrix (distances between depots):
[[ 0.   10.41  7.39  7.24 17.15]
 [10.41  0.    8.91  8.92 22.07]
 [ 7.39  8.91  0.   12.3  13.22]
 [ 7.24  8.92 12.3   0.   24.11]
 [17.15 22.07 13.22 24.11  0.  ]]


In [7]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# --- 1. Consolidate Data for the Solver ---
orders_df['Total_Demand'] = orders_df['Sterile_Kits_Demand'] + orders_df['Vaccine_Packs_Demand']

# <-- THE FIX IS HERE
NUM_VEHICLES = 25 # Increased from 10 to 25
VEHICLE_CAPACITY = 600

data = {}
data['distance_matrix'] = distance_matrix
data['demands'] = [0] * len(warehouses_df) + list(orders_df['Total_Demand'])
data['vehicle_capacities'] = [VEHICLE_CAPACITY] * NUM_VEHICLES
data['num_vehicles'] = NUM_VEHICLES
data['depot'] = 0


# --- 2. Function to Print the Solution ---
def print_solution(data, manager, routing, solution):
    print(f'Objective: {solution.ObjectiveValue() / 1000:.2f} miles')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = f'Route for vehicle {vehicle_id}:\n'
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += f' {node_index} (Load({route_load})) ->'
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        node_index = manager.IndexToNode(index)
        plan_output += f' {node_index} (Load({route_load}))\n'
        plan_output += f'Distance of the route: {route_distance / 1000:.2f} miles\n'
        plan_output += f'Load of the route: {route_load}\n'
        if route_load > 0:
            print(plan_output)
            total_distance += route_distance
            total_load += route_load
    print(f'Total distance of all routes: {total_distance / 1000:.2f} miles')
    print(f'Total load of all routes: {total_load}')

# --- 3. Set Up and Run the Solver ---
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)

def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return int(data['distance_matrix'][from_node][to_node] * 1000)

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

def demand_callback(from_index):
    from_node = manager.IndexToNode(from_index)
    return data['demands'][from_node]

demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index, 0, data['vehicle_capacities'], True, 'Capacity'
)

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.log_search = True
search_parameters.time_limit.FromSeconds(60)

# Solve the problem
print("Solving with an expanded fleet of 25 vehicles (60-second limit)...")
solution = routing.SolveWithParameters(search_parameters)

if solution:
    print("\n✅ Solution found!")
    print_solution(data, manager, routing, solution)
else:
    print('\nNo solution found in the time limit!')

Solving with an expanded fleet of 25 vehicles (60-second limit)...

✅ Solution found!
Objective: 145.49 miles
Route for vehicle 1:
 0 (Load(0)) -> 172 (Load(39)) -> 146 (Load(126)) -> 144 (Load(185)) -> 113 (Load(234)) -> 93 (Load(327)) -> 76 (Load(360)) -> 74 (Load(417)) -> 34 (Load(489)) -> 27 (Load(577)) -> 32 (Load(598)) -> 0 (Load(598))
Distance of the route: 3.68 miles
Load of the route: 598

Route for vehicle 6:
 0 (Load(0)) -> 175 (Load(32)) -> 48 (Load(76)) -> 60 (Load(124)) -> 139 (Load(173)) -> 153 (Load(250)) -> 185 (Load(315)) -> 9 (Load(373)) -> 55 (Load(424)) -> 169 (Load(515)) -> 161 (Load(593)) -> 0 (Load(593))
Distance of the route: 9.92 miles
Load of the route: 593

Route for vehicle 7:
 0 (Load(0)) -> 152 (Load(88)) -> 143 (Load(165)) -> 105 (Load(244)) -> 149 (Load(310)) -> 156 (Load(388)) -> 164 (Load(432)) -> 108 (Load(464)) -> 87 (Load(513)) -> 171 (Load(572)) -> 0 (Load(572))
Distance of the route: 4.63 miles
Load of the route: 572

Route for vehicle 8:
 0 (Loa

In [8]:
# --- Diagnostic Checks ---

print("--- Running Diagnostic Checks ---")

# Check 1: Is any single order larger than a vehicle's capacity?
max_demand = orders_df['Total_Demand'].max()
vehicle_capacity = VEHICLE_CAPACITY # This variable was defined in the solver cell

print(f"\n[Check 1: Individual Order Size]")
print(f"Largest single order demand: {max_demand} units")
print(f"Single vehicle capacity: {vehicle_capacity} units")

if max_demand > vehicle_capacity:
    print("🔴 RESULT: FAILED. At least one order is too big for any vehicle.")
else:
    print("✅ RESULT: PASSED. All individual orders can fit in a vehicle.")

# Check 2: Is the total demand greater than the whole fleet's capacity?
total_demand = orders_df['Total_Demand'].sum()
total_fleet_capacity = NUM_VEHICLES * vehicle_capacity # These were defined in the solver cell

print(f"\n[Check 2: Total Fleet Capacity]")
print(f"Total demand of all 200 orders: {total_demand} units")
print(f"Total capacity of all {NUM_VEHICLES} vehicles: {total_fleet_capacity} units")

if total_demand > total_fleet_capacity:
    print("🔴 RESULT: FAILED. The entire fleet is not large enough to handle all orders.")
else:
    print("✅ RESULT: PASSED. The fleet has enough total capacity for all orders.")

--- Running Diagnostic Checks ---

[Check 1: Individual Order Size]
Largest single order demand: 93 units
Single vehicle capacity: 600 units
✅ RESULT: PASSED. All individual orders can fit in a vehicle.

[Check 2: Total Fleet Capacity]
Total demand of all 200 orders: 11590 units
Total capacity of all 25 vehicles: 15000 units
✅ RESULT: PASSED. The fleet has enough total capacity for all orders.


In [9]:
import time
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# --- Parts 1 and 2 are unchanged ---
# --- 1. Consolidate Data for the Solver ---
orders_df['Total_Demand'] = orders_df['Sterile_Kits_Demand'] + orders_df['Vaccine_Packs_Demand']
NUM_VEHICLES = 25
VEHICLE_CAPACITY = 600
data = {}
data['distance_matrix'] = distance_matrix
data['demands'] = [0] * len(warehouses_df) + list(orders_df['Total_Demand'])
data['vehicle_capacities'] = [VEHICLE_CAPACITY] * NUM_VEHICLES
data['num_vehicles'] = NUM_VEHICLES
data['depot'] = 0

# --- 2. Function to Print the Solution ---
def print_solution(data, manager, routing, solution):
    # This function is unchanged
    print(f'Objective: {solution.ObjectiveValue() / 1000:.2f} miles')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = f'Route for vehicle {vehicle_id}:\n'
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += f' {node_index} (Load({route_load})) ->'
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        node_index = manager.IndexToNode(index)
        plan_output += f' {node_index} (Load({route_load}))\n'
        plan_output += f'Distance of the route: {route_distance / 1000:.2f} miles\n'
        plan_output += f'Load of the route: {route_load}\n'
        if route_load > 0:
            print(plan_output)
            total_distance += route_distance
            total_load += route_load
    print(f'Total distance of all routes: {total_distance / 1000:.2f} miles')
    print(f'Total load of all routes: {total_load}')

# --- CORRECTED Custom Search Monitor Class ---
class SolutionStagnationMonitor(pywrapcp.SearchMonitor):
    def __init__(self, routing, patience=180.0):
        super().__init__(routing.solver())
        self._routing = routing
        self._patience = patience
        self._best_objective = float('inf')
        self._last_improvement_time = None

    def EnterSearch(self):
        self._last_improvement_time = time.time()

    def AcceptSolution(self):
        current_objective = self._routing.CostVar().Value()
        
        if current_objective < self._best_objective:
            self._best_objective = current_objective
            self._last_improvement_time = time.time()
        
        elapsed_time_without_improvement = time.time() - self._last_improvement_time
        if elapsed_time_without_improvement > self._patience:
            print(f"\nStopping search. No improvement for {self._patience} seconds.")
            self.solver().FinishCurrentSearch()
        
        # --- THE FIX IS HERE ---
        # We must return True to tell the solver to continue.
        return True

# --- 3. Set Up and Run the Solver ---
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)

# (Callbacks are unchanged)
def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return int(data['distance_matrix'][from_node][to_node] * 1000)
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

def demand_callback(from_index):
    from_node = manager.IndexToNode(from_index)
    return data['demands'][from_node]
demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index, 0, data['vehicle_capacities'], True, 'Capacity'
)

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.log_search = True

# --- Using the Monitor Correctly ---
stagnation_monitor = SolutionStagnationMonitor(routing, patience=180)
routing.AddSearchMonitor(stagnation_monitor)

# Solve the problem
print("Solving with a 180-second stagnation limit...")
solution = routing.SolveWithParameters(search_parameters)

if solution:
    print("\n✅ Solution found!")
    print_solution(data, manager, routing, solution)
else:
    print('\nNo solution found!')

Solving with a 180-second stagnation limit...


SystemError: <built-in function SearchMonitor_AcceptDelta> returned a result with an exception set

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# --- 1. Extract the routes from the solution ---
print("Extracting routes from the solution...")
routes = []
for vehicle_id in range(data['num_vehicles']):
    route_for_vehicle = []
    index = routing.Start(vehicle_id)
    while not routing.IsEnd(index):
        node_index = manager.IndexToNode(index)
        route_for_vehicle.append(node_index)
        index = solution.Value(routing.NextVar(index))
    
    # Add the end node (depot) to complete the loop
    node_index = manager.IndexToNode(index)
    route_for_vehicle.append(node_index)
    
    # Only add routes that are actually used
    if len(route_for_vehicle) > 2:
        routes.append(route_for_vehicle)

print(f"✅ Found {len(routes)} active routes.")


# --- 2. Create the base map with depots and orders ---
print("Generating the final map with optimized routes...")
fig = px.scatter_mapbox(
    orders_df,
    lat="latitude",
    lon="longitude",
    hover_name="Zone",
    hover_data=["Order_ID", "Sterile_Kits_Demand", "Vaccine_Packs_Demand"],
    color_discrete_sequence=["blue"],
    zoom=9,
    height=600,
    title="Final Optimized Routes for Medical Supply Distribution"
)
fig.add_trace(
    px.scatter_mapbox(
        warehouses_df,
        lat="Latitude",
        lon="Longitude",
        hover_name="Warehouse_ID",
        color_discrete_sequence=["red"],
        size_max=20,
    ).data[0]
)


# --- 3. Add the route lines to the map ---
# We'll use a color scale to make each route a different color
color_scale = px.colors.qualitative.Plotly

for i, route in enumerate(routes):
    route_df = all_locations_df.iloc[route]
    fig.add_trace(
        go.Scattermapbox(
            mode = "lines",
            lon = route_df['lon'],
            lat = route_df['lat'],
            hoverinfo = 'none',
            line=dict(width=2, color=color_scale[i % len(color_scale)]),
            name=f"Vehicle Route {i}"
        )
    )

fig.update_layout(
    mapbox_style="open-street-map",
    margin={"r":0,"t":40,"l":0,"b":0},
    legend_title_text='Routes'
)

fig.show()

Extracting routes from the solution...
✅ Found 21 active routes.
Generating the final map with optimized routes...


C:\Users\abravikumar\AppData\Local\Temp\ipykernel_13344\3998160131.py:28: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

C:\Users\abravikumar\AppData\Local\Temp\ipykernel_13344\3998160131.py:40: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

C:\Users\abravikumar\AppData\Local\Temp\ipykernel_13344\3998160131.py:58: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [10]:
print("Starting pre-processing to create a lightweight data file for deployment...")

# --- Load Raw Data ---
# Ensure the large parquet and csv files are in your project folder to run this
df_trips = pd.read_parquet('yellow_tripdata_2024-01.parquet')
df_zones = pd.read_csv('taxi+_zone_lookup.csv')

# --- Embedded Coordinates Data ---
# Ensure the full 265 lines of coordinate data are pasted here
coords_csv_data = """LocationID,latitude,longitude
1,40.644990,-74.174230
2,40.675690,-73.800100
3,40.864470,-73.847420
4,40.723750,-73.981420
5,40.562860,-74.184470
6,40.596820,-74.126290
7,40.744110,-73.992010
8,40.627760,-73.951230
9,40.835430,-73.864330
10,40.652070,-73.945830
11,40.628880,-73.994870
12,40.588520,-73.963160
13,40.708870,-74.009380
14,40.668240,-73.980390
15,40.837330,-73.902260
16,40.764170,-73.961540
17,40.692690,-73.951560
18,40.695160,-73.922440
19,40.679610,-73.904830
20,40.648370,-74.017550
21,40.627270,-74.077330
22,40.671160,-73.943110
23,40.785340,-73.942450
24,40.791730,-73.944220
25,40.718530,-73.953560
26,40.641320,-73.955930
27,40.671190,-73.868720
28,40.743140,-73.974440
29,40.706850,-74.015240
30,40.545620,-74.166820
31,40.864820,-73.916100
32,40.852860,-73.896790
33,40.671810,-73.969180
34,40.686560,-73.981880
35,40.694860,-73.974280
36,40.711280,-73.994130
37,40.725830,-73.999060
38,40.887320,-73.910320
39,40.803770,-73.872720
40,40.820980,-73.889320
41,40.575910,-74.139090
42,40.612030,-74.025340
43,40.752690,-73.984280
44,40.714090,-73.942120
45,40.718450,-73.986870
46,40.738180,-74.002590
47,40.709320,-73.996120
48,40.768070,-73.982740
49,40.760160,-73.991250
50,40.757070,-73.990130
51,40.618030,-74.004230
52,40.740620,-73.913230
53,40.741710,-73.866330
54,40.796590,-73.839690
55,40.807750,-73.856980
56,40.689360,-73.899320
57,40.693150,-73.882420
58,40.840240,-73.856090
59,40.871140,-73.834140
60,40.896310,-73.854830
61,40.705290,-73.938390
62,40.748360,-73.953680
63,40.709350,-73.961290
64,40.761890,-73.826130
65,40.732890,-74.006930
66,40.707740,-73.951540
67,40.700010,-73.924230
68,40.761760,-73.958500
69,40.785070,-73.815560
70,40.814520,-73.910580
71,40.729990,-73.988020
72,40.696140,-73.994780
73,40.793830,-73.961560
74,40.799010,-73.968130
75,40.811740,-73.952230
76,40.702730,-74.014280
77,40.685370,-74.001220
78,40.658810,-73.974950
79,40.726480,-73.980590
80,40.695990,-73.969980
81,40.669560,-73.891890
82,40.679120,-73.961220
83,40.686030,-73.961910
84,40.635840,-73.884580
85,40.672580,-73.914270
86,40.687530,-73.906730
87,40.748640,-73.987730
88,40.759290,-73.985160
89,40.583610,-74.093950
90,40.763190,-73.991800
91,40.603330,-73.943990
92,40.598070,-74.076590
93,40.631890,-73.965560
94,40.599010,-74.062820
95,40.778840,-73.953530
96,40.784420,-73.949740
97,40.730470,-73.958810
98,40.608030,-74.088010
99,40.592750,-74.198390
100,40.751140,-73.992240
101,40.644120,-74.005180
102,40.638570,-73.994270
103,40.771980,-73.874430
104,40.775830,-73.869970
105,40.781060,-73.850540
106,40.728830,-73.978830
107,40.733610,-73.994230
108,40.751740,-73.800050
109,40.755450,-73.820520
110,40.735870,-73.791530
111,40.605280,-74.065870
112,40.612490,-73.978180
113,40.759160,-73.969240
114,40.764520,-73.978010
115,40.852170,-73.844910
116,40.743130,-73.992840
117,40.687050,-73.873230
118,40.835560,-73.850680
119,40.790070,-73.976050
120,40.738870,-73.976310
121,40.810330,-73.829040
122,40.825630,-73.811550
123,40.796390,-73.814230
124,40.790930,-73.853240
125,40.748970,-73.998930
126,40.767570,-73.805360
127,40.742970,-74.006590
128,40.735390,-73.993140
129,40.787620,-73.842040
130,40.759590,-73.844450
131,40.753340,-73.872970
132,40.774100,-73.980600
133,40.804890,-73.938830
134,40.755790,-73.883490
135,40.760030,-73.921350
136,40.766330,-73.858790
137,40.771230,-73.987540
138,40.718030,-73.843450
139,40.701150,-73.828550
140,40.771120,-73.957580
141,40.776930,-73.962240
142,40.767660,-73.971840
143,40.769930,-73.958560
144,40.743320,-73.982290
145,40.825130,-73.940340
146,40.827760,-73.931220
147,40.865730,-73.830490
148,40.755490,-73.976410
149,40.798650,-73.931180
150,40.815910,-73.940730
151,40.742300,-74.001600
152,40.745670,-73.993810
153,40.739480,-74.002450
154,40.746060,-73.811020
155,40.791550,-73.932720
156,40.873240,-73.865970
157,40.810560,-73.889370
158,40.749940,-73.987100
159,40.857640,-73.918970
160,40.868090,-73.899630
161,40.757040,-73.978270
162,40.763320,-73.973950
163,40.765970,-73.973210
164,40.761020,-73.968320
165,40.729090,-73.805560
166,40.739780,-73.990420
167,40.596040,-74.156640
168,40.795490,-73.940980
169,40.817360,-73.956960
170,40.756280,-73.962630
171,40.692370,-73.833330
172,40.584340,-74.106880
173,40.706110,-73.911380
174,40.692880,-73.867950
175,40.659910,-73.820980
176,40.601450,-74.098710
177,40.690850,-73.998010
178,40.680430,-73.993420
179,40.645480,-73.959380
180,40.652110,-73.964240
181,40.737110,-73.986300
182,40.844320,-73.880120
183,40.866160,-73.882700
184,40.826720,-73.859660
185,40.810000,-73.896780
186,40.752010,-73.993210
187,40.638760,-74.084260
188,40.669810,-73.929840
189,40.678460,-73.931750
190,40.757870,-73.969890
191,40.595930,-74.088670
192,40.583560,-74.072710
193,40.767120,-73.881240
194,40.697420,-73.808020
195,40.635870,-73.812970
196,40.709110,-73.867090
197,40.710730,-73.880480
198,40.724730,-73.880890
199,40.873530,-73.844160
200,40.659340,-73.796330
201,40.602730,-74.113330
202,40.715340,-74.004310
203,40.640690,-73.944230
204,40.551720,-74.204360
205,40.678520,-73.842780
206,40.642510,-73.848690
207,40.627040,-74.156090
208,40.771030,-73.905910
209,40.739330,-73.974460
210,40.730720,-73.991240
211,40.721930,-74.000160
212,40.686530,-73.834010
213,40.672040,-73.791520
214,40.584740,-74.057390
215,40.617540,-74.043810
216,40.617430,-74.092170
217,40.626880,-74.114780
218,40.573220,-74.098370
219,40.596660,-74.045790
220,40.577280,-74.067820
221,40.607480,-74.153920
222,40.655270,-73.991910
223,40.688140,-73.910300
224,40.749170,-74.001200
225,40.686250,-73.939160
226,40.710490,-73.983940
227,40.697620,-73.910790
228,40.758410,-73.919250
229,40.782340,-73.953790
230,40.760010,-73.988020
231,40.792290,-73.959930
232,40.797230,-73.951520
233,40.733560,-73.985930
234,40.749760,-73.991380
235,40.853040,-73.828230
236,40.775980,-73.953680
237,40.771820,-73.966230
238,40.754230,-73.973490
239,40.757900,-73.978250
240,40.840740,-73.939230
241,40.864750,-73.897440
242,40.860010,-73.871330
243,40.803520,-73.949600
244,40.807870,-73.945410
245,40.893120,-73.882340
246,40.740290,-73.994010
247,40.676380,-73.975390
248,40.669020,-73.961430
249,40.747480,-73.986220
250,40.728950,-73.992080
251,40.570000,-74.113060
252,40.851970,-73.928170
253,40.858700,-73.933330
254,40.876540,-73.909940
255,40.743990,-73.945930
256,40.734410,-73.954490
257,40.748350,-73.944370
258,40.718320,-73.908510
259,40.670350,-73.834920
260,40.702780,-73.883640
261,40.733310,-73.959830
262,40.753330,-73.982270
263,40.748360,-73.988180
264,0.000000,0.000000
265,0.000000,0.000000""" 
df_coords = pd.read_csv(io.StringIO(coords_csv_data))

# --- Perform Merging and Cleaning ---
df_trips_cleaned = df_trips[['PULocationID']].copy().rename(columns={'PULocationID': 'LocationID'})
df_merged = pd.merge(df_trips_cleaned, df_zones, on='LocationID')
df_final = pd.merge(df_merged, df_coords, on='LocationID')
df_final = df_final[(df_final['latitude'] != 0) & (df_final['Borough'] != 'Unknown')].copy()

# --- Sample and Save the Lightweight File ---
# We take a large sample so the app still has variety
pre_sampled_df = df_final.sample(n=10000, random_state=42)
pre_sampled_df.to_csv('orders_for_app.csv', index=False)

print("\n✅ Success! 'orders_for_app.csv' has been created.")
print("This lightweight file will be used by your deployed application.")

Starting pre-processing to create a lightweight data file for deployment...

✅ Success! 'orders_for_app.csv' has been created.
This lightweight file will be used by your deployed application.
